# Grid Search

This dataset is originally from the National Institute of Diabetes and Digestive and Kidney Diseases. *The objective is to predict based on diagnostic measurements whether a patient has diabetes.*

### Dataset information

[Pima Indians Diabetes](https://www.kaggle.com/uciml/pima-indians-diabetes-database)

Several constraints were placed on the selection of these instances from a larger database. In particular, all patients here are females at least 21 years old of Pima Indian heritage.

### Dataset description

* **Pregnancies**: Number of times pregnant

* **Glucose**: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

* **BloodPressure**: Diastolic blood pressure (mm Hg)

* **SkinThickness**: Triceps skin fold thickness (mm)

* **Insulin**: 2-Hour serum insulin (mu U/ml)

* **BMI**: Body mass index (weight in kg/(height in m)^2)

* **DiabetesPedigreeFunction**: Diabetes pedigree function

* **Age**: Age (years)

* **Outcome**: Class variable (0 or 1)

### Import Classes and Functions

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

### Load The Dataset

In [ ]:
df = pd.read_csv('../datasets/pima-indians-diabetes.csv')
df.head()

### Split into input (X) and output (Y) variables

In [ ]:
X = df[['Pregnancies', 'Glucose', 'BloodPressure',
               'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']]
y = df['Outcome']

# Keras and Tensorflow need numpy array as input
X = X.as_matrix()
Y = y.as_matrix()

### Define the Neural Network Model

In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(8, kernel_initializer='uniform'))
    model.add(Activation('relu'))
    model.add(Dense(1, kernel_initializer='uniform'))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### Create the Model

In [ ]:
model = KerasClassifier(build_fn=create_model, verbose=0)

### Define the grid search parameters

In [ ]:
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]

param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)

### Run the grid

In [ ]:
grid_result = grid.fit(X, Y)

### Summarize the results

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))